In [1]:
import ROOT, yaml
import numpy as np

Welcome to JupyROOT 6.30/07


In [2]:
def fit_signal(signal_mass=1000, remap={'mass_Zprime': 'mass_Zprime', 'weight': 'weight'}, SR='SR1'):
    f = ROOT.TFile(f'input/Run2/mc_signal_Hbb_{signal_mass}.root')
    tree = f.Get('Events')
    mass_Zprime = ROOT.RooRealVar(remap['mass_Zprime'], "mass_Zprime", signal_mass, 650, 4000)
    weight = ROOT.RooRealVar(remap['weight'], "weight", 0, -10, 10)
    mass_Higgs = ROOT.RooRealVar("mass_Higgs", "mass_Higgs", 125, 0, 999)
    tagger_Hbb = ROOT.RooRealVar("tagger_Hbb", "tagger_Hbb", 0, 0, 2)
    tagger_down, tagger_up = {'SR1': 0.7, 'SR2': 0.9}, {'SR1': 0.9, 'SR2': 2}
    SR_cut = f"(mass_Higgs>110) & (mass_Higgs<145) & (tagger_Hbb>{tagger_down[SR]}) & (tagger_Hbb<{tagger_up[SR]})"
    mc = ROOT.RooDataSet("signal_Hbb", "signal_Hbb", tree, ROOT.RooArgSet(mass_Zprime, weight, mass_Higgs, tagger_Hbb), SR_cut, remap['weight'])
    
    x0 = ROOT.RooRealVar("x0", "x0", signal_mass, signal_mass - 200, signal_mass + 200)
    sigmaL = ROOT.RooRealVar("sigmaL", "sigmaL", 50, 1e-8, 200)
    sigmaR = ROOT.RooRealVar("sigmaR", "sigmaR", 50, 1e-8, 200)
    alphaL = ROOT.RooRealVar("alphaL", "alphaL", 2, 1e-8, 6)
    alphaR = ROOT.RooRealVar("alphaR", "alphaR", 2, 1e-8, 6)
    nL = ROOT.RooRealVar("nL", "nL", 1, 1e-8, 50)
    nR = ROOT.RooRealVar("nR", "nR", 1, 1e-8, 50)

    model_signal = ROOT.RooCrystalBall("model_signal", "model_signal", mass_Zprime, x0, sigmaL, sigmaR, alphaL, nL, alphaR, nR)
    model_signal.fitTo(mc, ROOT.RooFit.SumW2Error(True))

    """
    print("x0:", x0.getVal())
    print("sigmaL:", sigmaL.getVal())
    print("sigmaR:", sigmaR.getVal())
    print("alphaL:", alphaL.getVal())
    print("alphaR:", alphaR.getVal())
    print("nL:", nL.getVal())
    print("nR:", nR.getVal())
    """
    return x0.getVal(), sigmaL.getVal(), sigmaR.getVal()

In [3]:
parameters = {}
uncertainties = {}
for SR in ['SR1', 'SR2']:
    uncertainties[SR] = {}
    for m in [700, 800, 900, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 3000, 3500]:
        parameters['nominal'] = fit_signal(signal_mass=m, SR=SR)
        parameters['JES_up'] = fit_signal(signal_mass=m, remap={'mass_Zprime': 'mass_Zprime_JES_up', 'weight': 'weight'}, SR=SR)
        parameters['JES_down'] = fit_signal(signal_mass=m, remap={'mass_Zprime': 'mass_Zprime_JES_down', 'weight': 'weight'}, SR=SR)
        parameters['JER_up'] = fit_signal(signal_mass=m, remap={'mass_Zprime': 'mass_Zprime_JER_up', 'weight': 'weight'}, SR=SR)
        parameters['JER_down'] = fit_signal(signal_mass=m, remap={'mass_Zprime': 'mass_Zprime_JER_down', 'weight': 'weight'}, SR=SR)
        parameters['PU_up'] = fit_signal(signal_mass=m, remap={'mass_Zprime': 'mass_Zprime', 'weight': 'weight_PU_up'}, SR=SR)
        parameters['PU_down'] = fit_signal(signal_mass=m, remap={'mass_Zprime': 'mass_Zprime', 'weight': 'weight_PU_down'}, SR=SR)

        uncertainties[SR][m] = {
            'x0': {
                k: (parameters[f'{k}_up'][0] - parameters[f'{k}_down'][0])/parameters['nominal'][0]/2
                for k in ['JES', 'JER', 'PU']
            },
            'sigmaL': {
                k: (parameters[f'{k}_up'][1] - parameters[f'{k}_down'][1])/parameters['nominal'][1]/2
                for k in ['JES', 'JER', 'PU']
            },
            'sigmaR': {
                k: (parameters[f'{k}_up'][2] - parameters[f'{k}_down'][2])/parameters['nominal'][2]/2
                for k in ['JES', 'JER', 'PU']
            }
        }

[#1] INFO:DataHandling -- RooAbsReal::attachToTree(mass_Zprime) TTree Float_t branch mass_Zprime will be converted to double precision.
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(mass_Higgs) TTree Float_t branch mass_Higgs will be converted to double precision.
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(tagger_Hbb) TTree Float_t branch tagger_Hbb will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(signal_Hbb) Skipping event #23 because mass_Zprime cannot accommodate the value 638.779
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(signal_Hbb) Skipping event #24 because mass_Zprime cannot accommodate the value 646.491
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(signal_Hbb) Skipping event #39 because mass_Zprime cannot accommodate the value 636.215
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(signal_Hbb) Skipping event #44 because mass_Zprime cannot accommodate the value 639.401
[#1] INFO:DataHandling -

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       350.8378374 Edm =       26.55669699 NCalls =     27
Info in <Minuit2>: NegativeG2LineSearch Doing a NegativeG2LineSearch since one of the G2 component is negative
Info in <Minuit2>: MnSeedGenerator Negative G2 found - new state: 
  Minimum value : 350.8378374
  Edm           : 26.55669699
  Internal parameters:	[    -0.3398369118    -0.3398369118     -1.287002219     -1.287002219    -0.5235987757    -0.5235987757                0]	
  Internal gradient  :	[                0     -54.64122164                0     -111.6478354    -0.3070873252      48.84878554      44.48755021]	
  Internal covariance matrix:
[[              2              0              0              0              0              0              0]
 [              0   0.0076762054              0              0              0              0              0]
 [              0      

In [9]:
with open('../../src/parameters/uncertainty/shape_uncertainties.yaml', 'w', encoding='utf-8') as f:
    yaml.dump(uncertainties, f)

In [ ]:
year = 'Run2'
fit_range_down, fit_range_up = 650, 4000
tagger_cut_low, tagger_cut_high = 0.9, 2

bkg_model_dir = f'output/{year}/background'

# # Background modelling
f = ROOT.TFile(f"input/{year}/data_Hbb.root", "r")
# Load TTree
tree = f.Get("Events")

# Define mass and weight variables
mass_Zprime = ROOT.RooRealVar("mass_Zprime", "mass_Zprime", 1500, fit_range_down, fit_range_up)
weight = ROOT.RooRealVar("weight", "weight", 1, -10, 10)
mass_Higgs = ROOT.RooRealVar("mass_Higgs", "mass_Higgs", 125, 0, 999)
tagger_Hbb = ROOT.RooRealVar("tagger_Hbb", "tagger_Hbb", 0, 0, 2)

SR_cut = f"(mass_Higgs>110) & (mass_Higgs<145) & (tagger_Hbb>{tagger_cut_low}) & (tagger_Hbb<{tagger_cut_high})"
sideband_cut = f"""
(
    ((mass_Higgs>50) & (mass_Higgs<70)) | 
    ((mass_Higgs>100) & (mass_Higgs<110)) | 
    (mass_Higgs>145)
) & 
(tagger_Hbb>{tagger_cut_low}) & (tagger_Hbb<{tagger_cut_high})
"""

data_SR = ROOT.RooDataSet("data_SR", "data_SR", tree, ROOT.RooArgSet(mass_Zprime, weight, mass_Higgs, tagger_Hbb), SR_cut, "weight")
data_sideband = ROOT.RooDataSet("data_sideband", "data_sideband", tree, ROOT.RooArgSet(mass_Zprime, weight, mass_Higgs, tagger_Hbb), sideband_cut, "weight")


In [ ]:
data_sideband.sumEntries(), data_SR.sumEntries()